<a href="https://colab.research.google.com/github/filsto/DigitRecognizer/blob/main/Digit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split 

In [9]:
# Partie A - les données

from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [11]:
#loader les datasets
train_set = pd.read_csv('/content/drive/MyDrive/DATA/Digit_recognizer/train.csv', sep=",")
test_set = pd.read_csv('/content/drive/MyDrive/DATA/Digit_recognizer/test.csv', sep=",")

In [ ]:
#préparer les datas
y_train=(train_set['label'])/255
X_train=(train_set.drop(['label'], axis = 1))/255

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)

#redimensionner X
X_train = X_train.values.reshape(-1,28,28,1)
X_test = test_set.values.reshape(-1,28,28,1)

#categoriser y
y_train=pd.get_dummies(y_train)

#splitter
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=0)

In [ ]:
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

In [15]:
# Partie B - le réseau de neurones

modele = keras.Sequential([
                  layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28,28,1)),
                  layers.MaxPool2D(),

                  layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu'),
                  layers.MaxPool2D(),

                  layers.Flatten(),

                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(1024, activation='relu'),
                  layers.Dropout(0.1),         
                  layers.Dense(1024, activation='relu'),
                  layers.Dense(10, activation='softmax')       
])

print(modele.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 14, 14, 16)        4624      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 1024)             

In [27]:
modele.compile(
    optimizer = 'Adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# Partie C - entraînement

history = modele.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid))


985/985 [==============================] - 147s 150ms/step - loss: 0.0162 - accuracy: 0.9968 - val_loss: 0.5821 - val_accuracy: 0.9839
Epoch 110/200
Epoch 110/200
816/985 [=======================>......] - ETA: 23s - loss: 0.0058 - accuracy: 0.9989

In [ ]:
history_data = pd.DataFrame(history.history)
history_data.plot(figsize=(8,5))
plt.grid=True
plt.gca().set_ylim(0,1) #regler la plage verticale sur [0-1]

In [ ]:
history_data['loss','val_loss'].plot()
#history_data[5:,['loss','val_loss']].plot()
history_data['accuracy','val_accuracy'].plot()

print(('best validation loss: {:0.4f}'+\
    '\nbest validation accuracy: {:0.4f}')\
    .format(history_data['val_loss'].min(),
            history_data['binary_accuracy'].max()))

In [ ]:
# Partie D - prédiction
y_test = modele.predict(X_test)

In [ ]:
# Partie E - sauvegarde

